In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import scipy
import os
import sys
from glob import glob
import cv2 
import time
import datetime

from shutil import copy
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications.efficientnet import EfficientNetB0 as  PretrainedModel,preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# all_images = glob("./images/*/*.jpg")

# print(all_images[1])

./images/position/position_31.jpg


In [7]:
# for im in all_images:
#     im_tmp = cv2.imread(im)[50:100, 150:220]
#     cv2.imwrite(im, im_tmp)

In [90]:
# for idx in range(1,10,1):
#     os.makedirs(f"../digit_recog/images/{idx}/")

In [94]:
# for idx in range(1,10,1):
#     copy(f"../images/references/{idx}.jpg", f"../digit_recog/images/{idx}/{idx}.jpg")

In [98]:
# for im in images:
#     for i in im:
#         for idx in range(100):
#             im_tmp = i.replace(".jpg",f"_{idx}.jpg")
#             copy(i, im_tmp)

In [13]:
ptm = PretrainedModel(
        input_shape = [100,100,3],
        weights = 'imagenet',
        include_top = False

)

2023-01-08 16:39:04.027767: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
ptm.trainable = False
K = len(folders)

x = Flatten()(ptm.output)
x = Dense(K, activation = 'softmax')(x)

In [15]:
model = Model(inputs = ptm.input , outputs = x)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 100, 100, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 100, 100, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 101, 101, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

In [16]:
gen = ImageDataGenerator(
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        shear_range = 0.1,
        zoom_range = 0.2,
        horizontal_flip = False,
        preprocessing_function = preprocess_input
)


In [17]:
batch_size = 64


In [18]:
IMAGE_SIZE = [100,100]

In [19]:
train_generator = gen.flow_from_directory(      
    "../digit_recog/images",
    shuffle = True,
    target_size = IMAGE_SIZE,
    batch_size = batch_size
    )

test_generator = gen.flow_from_directory(
    "../digit_recog/images",
    target_size = IMAGE_SIZE,
    batch_size = batch_size
)


Found 1212 images belonging to 12 classes.
Found 1212 images belonging to 12 classes.


In [20]:
model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])


In [21]:
acc_thresh = 0.985
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('accuracy') > acc_thresh):
            print("Stopping training as desired accuracy reached....")
            self.model.stop_training = True


In [22]:
myCall = myCallback()

In [23]:
image_files = glob("../digit_recog/images/*/*.jpg")
test_image_files = glob("../digit_recog/images/*/*.jpg")

In [24]:
r = model.fit(
        train_generator,
        validation_data = test_generator,
        epochs = 2,
        steps_per_epoch = int(np.ceil(len(image_files)/batch_size)),
        validation_steps = int(np.ceil(len(test_image_files)/batch_size)),
        callbacks=[myCall]
)

2023-01-08 16:39:51.023766: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/2
19/19 [==============================] - 29s 1s/step - loss: 0.3246 - accuracy: 0.9026 - val_loss: 8.2902e-04 - val_accuracy: 1.0000
Epoch 2/2
19/19 [==============================] - 18s 985ms/step - loss: 0.0025 - accuracy: 0.9992 - val_loss: 2.1664e-05 - val_accuracy: 1.0000
Stopping training as desired accuracy reached....


In [35]:
model.save("./models/model.h5")

/Users/raprak-blrm20/opt/anaconda3/envs/RL/lib/python3.9/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [36]:
image = cv2.imread("/Users/raprak-blrm20/Desktop/personal/CODES/m-projects/a-games/wallsmash_cracker/images/boxes8/patch_12.jpg")[50:100, 150:220]
image = cv2.resize(image, (100,100))
image = image.reshape(1,100,100,3)
np.argmax(model.predict(image))

9

In [31]:
model = load_model("./models/model.h5")